In [ ]:
import os
import shutil

In [ ]:
raw_data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri'

bids_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'

subnums = ['601', '609', '611', '619', '621', '629']

sessions = ['01', '02', '03']

data_types = ['anat', 'func']

misc_files = ['dataset_description.json', 'participants.json', 'README', 'CHANGES', 'LICENSE']

In [ ]:
from bidsify_helpers import make_bids_dirs, add_misc_bids_files

In [ ]:
make_bids_dirs()

In [ ]:
add_misc_bids_files()

In [ ]:
import nibabel as nib

In [ ]:
img = nib.load('/Users/zeynepenkavi/Downloads/lombardi_overtrained_decisions_fmri/sub-45087/ses-30994/func/sub-45087_ses-30994_task-fmri_acq-3_rec-1_run-1_bold.nii.gz')

In [ ]:
dict(img.header)

In [ ]:
cur_sub = '601'
cur_ses = '01'
raw_path_ = raw_data_path
cur_raw_dir = os.path.join(raw_path_, 'sub-'+cur_sub, 'ses-'+cur_ses)

In [ ]:
bids_path_ = bids_path
cur_bids_dir = os.path.join(bids_path_, 'sub-'+cur_sub, 'ses-'+cur_ses, 'func')

In [ ]:
if not (os.path.exists(cur_bids_dir)):
    print('Creating bids path: ' + cur_bids_dir)
    os.makedirs(cur_bids_dir)

In [ ]:
cur_raw_files = os.listdir(cur_raw_dir)
cur_raw_files

In [ ]:
func_key_ = 'fmri'
[i for i in cur_raw_files if func_key_ in i]

In [ ]:
cur_raw_files = os.listdir(cur_raw_dir)
cur_func_files = [i for i in cur_raw_files if func_key_ in i]
for cur_func_file in cur_func_files:
    shutil.copy(os.path.join(cur_raw_dir, cur_func_file), os.path.join(cur_bids_dir, cur_func_file))

In [ ]:
bids_path

In [ ]:
import glob

In [ ]:
all_func_sidecars = glob.glob(bids_path_ + '/*/*/func/*_bold.json')

In [ ]:
os.path.basename(all_func_sidecars[0])

In [ ]:
import json

In [ ]:
f = open(all_func_sidecars[0],"r")
data = f.read()
tmp = json.loads(data)

In [ ]:
tmp

In [ ]:
SliceTiming = [0.0, 0.05833, 0.11666, 0.17499, 0.23332, 0.29165, 0.34998, 0.40831, 0.46664, 0.52497, 0.5833, 0.64163, 0.69996, 0.75829, 0.81662, 0.87495, 0.93328, 0.99161, 1.04994, 1.10827, 1.1666, 1.22493, 1.28326, 1.34159, 1.39992, 1.45825, 1.51658, 1.57491, 1.63324, 1.69157, 1.7499, 1.80823, 1.86656, 1.92489, 1.98322, 2.04155, 2.09988, 2.15821, 2.21654, 2.27487, 2.3332, 2.39153]

In [ ]:
add_fields = {'yesNo': {'TaskName': 'yesNo', 'SliceTiming': SliceTiming}, 'binaryChoice': {'TaskName': 'binaryChoice', 'SliceTiming': SliceTiming}}

In [ ]:
# cur_task = ...
if 'yesNo' in os.path.basename(all_func_sidecars[0]):
    cur_task = 'yesNo'
else:
    cur_task = 'binaryChoice'
    

In [ ]:
tmp.update(add_fields[cur_task])

In [ ]:
tmp

In [ ]:
tempfile = os.path.join(os.path.dirname(all_func_sidecars[0]), 'tmp.json')
with open(tempfile, 'w') as f:
    json.dump(tmp, f, indent=4)

# rename temporary file replacing old file
# os.rename(tempfile, filename)

In [ ]:
all_func_sidecars[0]

In [ ]:
os.rename(tempfile, all_func_sidecars[0])

In [ ]:
'SliceTiming' in tmp.keys()

In [ ]:
import random

In [ ]:
random.randint(0, 1000)

In [ ]:
'tmp'+str(random.randint(0, 1000))+'.json'

In [ ]:
os.path.join(os.path.dirname(all_func_sidecars[0]), 'tmp'+str(random.randint(0, 1000))+'.json')

In [ ]:
all_func_sidecars = glob.glob(bids_path_ + '/*/*/func/*_bold.json')

for cur_sidecar in all_func_sidecars:
    f = open(cur_sidecar,"r")
    data = f.read()
    tmp = json.loads(data)

    if 'SliceTiming' in tmp.keys():
        print('Sidecar contains SliceTiming. Moving onto next. (%s)' %(os.path.basename(cur_sidecar)))
        continue

    else:
        if 'yesNo' in os.path.basename(cur_sidecar):
            cur_task = 'yesNo'
        else:
            cur_task = 'binaryChoice'
        print('Determined task name for %s as %s'%(os.path.basename(cur_sidecar), cur_task))

        print('Updating sidecar...')
        tmp.update(add_fields[cur_task])

        print('Saving updated sidecar...')
        tempfile = os.path.join(os.path.dirname(cur_sidecar), 'tmp'+str(random.randint(0, 1000))+'.json')
        with open(tempfile, 'w') as f:
            json.dump(tmp, f, indent=4)

        # rename temporary file replacing old file
        print('Replacing old sidecar...')
        os.rename(tempfile, cur_sidecar)